In [3]:
from pathlib import Path
import numpy as np
import IPython.display as ipd
from simple_mp.simple import simple_fast
from tqdm import tqdm
import pandas as pd

root = Path("../data")
motif = root / "motif/train_short_audio"

data = []
for path in tqdm(motif.glob("**/motif.cens.*.npy")):
    data.append(dict(
        data=np.load(path),
        species=path.parent.parent.name,
        example_name=path.parent.name,
        motif_name=path.name
    ))

1002it [00:00, 1941.68it/s]


In [6]:
df = pd.DataFrame(data)
df.head()

,data,species,example_name,motif_name
0,"[[0.3756666878584689, 0.37259816530431233, 0.3...",acafly,XC109605,motif.cens.0.npy
1,"[[0.3526420750609852, 0.35178527616127747, 0.3...",acafly,XC109605,motif.cens.1.npy
2,"[[0.3247211285736167, 0.3227699842406104, 0.31...",acafly,XC11209,motif.cens.0.npy
3,"[[0.322090386851785, 0.3195279306817608, 0.317...",acafly,XC11209,motif.cens.1.npy
4,"[[0.28324403789352703, 0.2856009155957204, 0.2...",acafly,XC127032,motif.cens.0.npy


In [42]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df.species)
y = le.transform(df.species)
y.shape

(1002,)

In [53]:
df.shape

(1002, 4)

In [54]:
X = np.concatenate(df.data.values).ravel().reshape(df.shape[0], -1)
X.shape

(1002, 600)

In [66]:
np.pad(X, [(0, 0), (0, 25)])

array([[0.37566669, 0.37259817, 0.36902592, ..., 0.        , 0.        ,
        0.        ],
       [0.35264208, 0.35178528, 0.35078298, ..., 0.        , 0.        ,
        0.        ],
       [0.32472113, 0.32276998, 0.31996407, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.37417984, 0.37610762, 0.37730375, ..., 0.        , 0.        ,
        0.        ],
       [0.30364935, 0.30521215, 0.30644792, ..., 0.        , 0.        ,
        0.        ],
       [0.25882975, 0.25626867, 0.25396012, ..., 0.        , 0.        ,
        0.        ]])

In [75]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    np.pad(X, [(0, 0), (0, 25)]), y, test_size=0.8
)

In [76]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset =tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [86]:
def get_compiled_model():
    model = tf.keras.Sequential([
        layers.Reshape((25, 25, 1), input_shape=(625,)),
        layers.Conv2D(96, (5, 5), activation='relu', input_shape=(25, 25, 1)),
        layers.MaxPooling2D((3, 3)),
        layers.Conv2D(192, (5, 5), activation='relu'),
        layers.MaxPooling2D((3, 3)),
        layers.Conv2D(192, (3, 3), activation='relu'),
        layers.Conv2D(192, (1, 1), activation='relu'),
        layers.Conv2D(len(le.classes_), (1, 1), activation='relu'),
        layers.Softmax(),
    ])

    model.compile(optimizer='adam',
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=['accuracy'])
    return model

In [87]:
model = get_compiled_model()
history = model.fit(train_dataset, epochs=10)

ValueError: Negative dimension size caused by subtracting 3 from 1 for '{{node conv2d_22/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, conv2d_22/Conv2D/ReadVariableOp)' with input shapes: [?,1,1,192], [3,3,192,192].